In [1]:
import os

import shutil

from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

os.chdir("/content/drive/MyDrive/PROJET_CALEDONIE/test_RCNN")

Mounted at /content/drive/


In [2]:
""" Répertoires source """

src_paths = ["/content/drive/MyDrive/PROJET_CALEDONIE/test_RCNN/AI4Mankind_refined_dataset",
             "/content/drive/MyDrive/PROJET_CALEDONIE/test_RCNN/Fausses_fumées_dataset",
             "/content/drive/MyDrive/PROJET_CALEDONIE/test_RCNN/Annotations_manuelles",
             "/content/drive/MyDrive/PROJET_CALEDONIE/test_RCNN/inceptionV3_annotated",
             "/content/drive/MyDrive/PROJET_CALEDONIE/test_RCNN/annotation_ben"]

""" Répertoires cible """

train_path = "/content/drive/MyDrive/PROJET_CALEDONIE/test_RCNN/train"
val_path = "/content/drive/MyDrive/PROJET_CALEDONIE/test_RCNN/val"
test_path = "/content/drive/MyDrive/PROJET_CALEDONIE/test_RCNN/test"
target_paths = {"train": train_path, "val": val_path, "test": test_path}

In [4]:
choice = "N"

while choice != "Y":        # entrée des quantités d'images à prendre des différents dossiers source
    print("\b"*1000)
    nb_img_per_src = []
    for path in src_paths:
        dir = os.listdir(path+"/image")
        length = len(dir)
        quant = input("Nombre d'images provenant de "+path.split(sep="/")[-1]+"("+str(length)+")"+": ").lower()
        if quant == "max":
            quant = length
        quant = int(quant)

        while quant > length:
            print("---> Pas assez d'images.")
            quant = input("Nombre d'images provenant de "+path.split(sep="/")[-1]+"("+str(length)+")"+": ").lower()
            if quant == "max":
                quant = length
            quant = int(quant)

        nb_img_per_src.append(quant)
    choice = input("\n===> Le dataset final comptera "+str(sum(nb_img_per_src))+" images. Poursuivre ? (Y/N): ").upper()

choice = "N"
print()
while choice != "Y":        # entrée de la répartition des images du dataset dans les catégories train, val et test
    ratio_train_val_test = {"train": 0, "val": 0, "test": 0}
    for target in ratio_train_val_test:
        percent = -1
        while (percent < 0) or (sum([ratio_train_val_test[cat] for cat in ratio_train_val_test]) > 1):
            percent = float(input("Pourcentage du dataset en {0}: ".format(target.upper())))
            ratio_train_val_test[target] = percent
            # print(percent, sum([ratio_train_val_test[cat] for cat in ratio_train_val_test]))
    choice = input("\n===> Il y aura donc {0}% en TRAIN, {1}% en VALIDATION et {2}% en TEST. Poursuivre ? (Y/N): ".format(ratio_train_val_test["train"]*100, ratio_train_val_test["val"]*100, ratio_train_val_test["test"]*100)).upper()


import random
import xml.etree.ElementTree as ET
import PIL

log = {"PNG": 0, "JPG": 0, "OTHER": 0}
print()

if set(os.listdir(train_path)) != {"image", "xml"}:
    print("Creating image and xml dir in train dataset")
    os.mkdir(train_path+"/image")
    os.mkdir(train_path+"/xml")
if set(os.listdir(val_path)) != {"image", "xml"}:
    print("Creating image and xml dir in val dataset")
    os.mkdir(val_path+"/image")
    os.mkdir(val_path+"/xml")
if set(os.listdir(test_path)) != {"image", "xml"}:
    print("Creating image and xml dir in test dataset")
    os.mkdir(test_path+"/image")
    os.mkdir(test_path+"/xml")

id = 1000
index = 0
for path in src_paths:
    print("\n-> Extracting "+str(nb_img_per_src[index])+" images from "+path.split(sep="/")[-1]+"...")
    image_list = os.listdir(path+"/image")
    image_list = image_list[:nb_img_per_src[index]]
    length = len(image_list)
    random.shuffle(image_list)
    # print(image_list)

    partition = [image_list[:int(length*ratio_train_val_test["train"])], 
                 image_list[int(length*ratio_train_val_test["train"]):int(length*(ratio_train_val_test["train"]+ratio_train_val_test["val"]))],
                 image_list[int(length*(ratio_train_val_test["train"]+ratio_train_val_test["val"])):]]
    i = 0     
    for cat in ratio_train_val_test:
        image_list = partition[i]
        print("--> {0} images en {1}...".format(len(image_list), cat.upper()))
        
        for image in image_list:
            l = image.split(sep='.')
            l, ext = l[:-1], l[-1]
            image_no_extension = ""
            for s in l:
                image_no_extension += ('.'+s)
            image_no_extension = image_no_extension[1:]
            xml_name = image_no_extension+".xml"

            # print(xml_name)
            try:                # Si le .xml correspondant existe, on le modifie
                tree = ET.parse(path+"/xml/"+xml_name)
                root_xml = tree.getroot()
                for filename in root_xml.findall('filename'):
                    filename.text = image
                for chemin in root_xml.findall('path'):
                    chemin.text = cat+'/'+image
                for folder in root_xml.findall('folder'):
                    folder.text = 'image'
                tree.write(target_paths[cat]+"/xml/"+str(id)+".xml")
                # shutil.copyfile(path+"/xml/"+xml_name, target_paths[cat]+"/xml/"+str(id)+".xml")

                if ext == "PNG" or ext == "png":                # if the image is a .png => convert into jpeg  (with conversion from rgba to rgb if necessary)
                    rgba = PIL.Image.open(path+"/image/"+image)
                    # try:    #if rgba, converting. if not rgba format, direct save
                    rgb = rgba.convert('RGB')
                    # except:
                    #     pass
                    rgb.save(target_paths[cat]+"/image/"+str(id)+".jpeg")
                    log["PNG"] += 1
                elif ext in {"jpg", "jpeg", "JPG"}:             # if the image is a .jpeg => direct transfert
                    shutil.copyfile(path+"/image/"+image, target_paths[cat]+"/image/"+str(id)+".jpeg")
                    log["JPG"] += 1
                else:                                           # if other extension => IGNORE
                    print("!! " + image + "   => extension not supported.")
                    log["OTHER"] += 1


                id += 1
            except FileNotFoundError:
                print("!! Fichier .xml manquant pour " + image)
                pass
        i += 1

    index += 1

print("Transfert terminé.", log)
# TODO : convert png in jpeg



In [ ]:
import random
import xml.etree.ElementTree as ET
import PIL

log = {"PNG": 0, "JPG": 0, "OTHER": 0}
print()

if set(os.listdir(train_path)) != {"image", "xml"}:
    print("Creating image and xml dir in train dataset")
    os.mkdir(train_path+"/image")
    os.mkdir(train_path+"/xml")
if set(os.listdir(val_path)) != {"image", "xml"}:
    print("Creating image and xml dir in val dataset")
    os.mkdir(val_path+"/image")
    os.mkdir(val_path+"/xml")
if set(os.listdir(test_path)) != {"image", "xml"}:
    print("Creating image and xml dir in test dataset")
    os.mkdir(test_path+"/image")
    os.mkdir(test_path+"/xml")

id = 1000
index = 0

random.seed(4) # Change to have different dataset generation

for path in src_paths:
    print("\n-> Extracting "+str(nb_img_per_src[index])+" images from "+path.split(sep="/")[-1]+"...")
    image_list = os.listdir(path+"/image")
    image_list = image_list[:nb_img_per_src[index]]
    length = len(image_list)
    random.shuffle(image_list)
    # print(image_list)

    partition = [image_list[:int(length*ratio_train_val_test["train"])], 
                 image_list[int(length*ratio_train_val_test["train"]):int(length*(ratio_train_val_test["train"]+ratio_train_val_test["val"]))],
                 image_list[int(length*(ratio_train_val_test["train"]+ratio_train_val_test["val"])):]]
    i = 0     
    for cat in ratio_train_val_test:
        image_list = partition[i]
        print("--> {0} images en {1}...".format(len(image_list), cat.upper()))
        
        for image in image_list:
            l = image.split(sep='.')
            l, ext = l[:-1], l[-1]
            image_no_extension = ""
            for s in l:
                image_no_extension += ('.'+s)
            image_no_extension = image_no_extension[1:]
            xml_name = image_no_extension+".xml"

            # print(xml_name)
            try:                # Si le .xml correspondant existe, on le modifie
                tree = ET.parse(path+"/xml/"+xml_name)
                root_xml = tree.getroot()
                for filename in root_xml.findall('filename'):
                    filename.text = image
                for chemin in root_xml.findall('path'):
                    chemin.text = cat+'/'+image
                for folder in root_xml.findall('folder'):
                    folder.text = 'image'
                tree.write(target_paths[cat]+"/xml/"+str(id)+".xml")
                # shutil.copyfile(path+"/xml/"+xml_name, target_paths[cat]+"/xml/"+str(id)+".xml")

                if ext == "PNG" or ext == "png":                # if the image is a .png => convert into jpeg  (with conversion from rgba to rgb if necessary)
                    rgba = PIL.Image.open(path+"/image/"+image)
                    # try:    #if rgba, converting. if not rgba format, direct save
                    rgb = rgba.convert('RGB')
                    # except:
                    #     pass
                    rgb.save(target_paths[cat]+"/image/"+str(id)+".jpeg")
                    log["PNG"] += 1
                elif ext in {"jpg", "jpeg", "JPG"}:             # if the image is a .jpeg => direct transfert
                    shutil.copyfile(path+"/image/"+image, target_paths[cat]+"/image/"+str(id)+".jpeg")
                    log["JPG"] += 1
                else:                                           # if other extension => IGNORE
                    print("!! " + image + "   => extension not supported.")
                    log["OTHER"] += 1


                id += 1
            except FileNotFoundError:
                print("!! Fichier .xml manquant pour " + image)
                pass
        i += 1

    index += 1

print("Transfert terminé.", log)
# TODO : convert png in jpeg



-> Extracting 860 images from AI4Mankind_refined_dataset...
--> 688 images en TRAIN...
--> 172 images en VAL...
--> 0 images en TEST...

-> Extracting 82 images from Fausses_fumées_dataset...
--> 65 images en TRAIN...
--> 17 images en VAL...
--> 0 images en TEST...

-> Extracting 100 images from Annotations_manuelles...
--> 80 images en TRAIN...
!! Fichier .xml manquant pour fire-9947.62502763.png
--> 20 images en VAL...
!! Fichier .xml manquant pour 103 feux de forêt actifs dans le Nord-Ouest.ca.webp
--> 0 images en TEST...

-> Extracting 310 images from inceptionV3_annotated...
--> 248 images en TRAIN...
!! Fichier .xml manquant pour inceptionV3_213.jpg
--> 62 images en VAL...
--> 0 images en TEST...

-> Extracting 163 images from annotation_ben...
--> 130 images en TRAIN...
!! Fichier .xml manquant pour 48059539.jpg
!! Fichier .xml manquant pour 47548154.jpg
!! Fichier .xml manquant pour 87542580.jpg
!! Fichier .xml manquant pour 77175695.jpg
!! Fichier .xml manquant pour 3369090